## Notes
1. Require config.json

    {
	    "api_key" : "xxxxx",
	    "server" : "https://data.reg.cmu.ac.th",
		"csv_file" : "ckan_data.csv"
    }

## 0 Install required packages

In [ ]:
import sys
!{sys.executable} -m pip install pandas

## 1 Load libraries and configurations

In [52]:
import json
import pprint
import urllib
import pandas as pd
import requests

# read configuration
config_file = 'config.json'
config = json.load(open(config_file))
api_key = config['api_key']
server = config['server']
csv_file = config['csv_file']


## 2 Function to submit request to server

In [ ]:
def send_request_to_server(server, api_key, action_path, dataset_dict):
	headers_dict = {
		'Authorization': api_key,
		'Content-Type' : 'application/x-www-form-urlencoded'
	}
	if dataset_dict != None:
		dataset_dict = urllib.parse.quote(json.dumps(dataset_dict))
	else:
		dataset_dict = []
	req = requests.post(server + 'api/action/' + action_path, headers=headers_dict, data=dataset_dict)
	
	# Use the json module to load CKAN's response into a dictionary.
	return req.json()

## 3 Functions to handling CKAN

In [ ]:
def create_package(server, api_key, dataset_dict):
	send_request_to_server(server, api_key, 'package_create', dataset_dict)

In [ ]:
def update_package(server, api_key, dataset_dict):
	send_request_to_server(server, api_key, 'package_update', dataset_dict)

In [ ]:
def get_package_info(server, api_key, package_name):
	return send_request_to_server(server, api_key, 'package_show?id=' + package_name, None)

In [ ]:
def get_resource_search_name_name(server, api_key, resource_name):
	response = send_request_to_server(server, api_key, 'resource_search?query=name:' + resource_name, None)
	if response['result']['count'] > 0:
		return response['result']['results'][0]
	return None

In [ ]:
def create_resource(server, api_key, dataset_dict):
	package_info = get_package_info(server, api_key, dataset_dict['name'])
	package_id = package_info['result']['id']
	dataset_dict_resource = {
		'package_id' : package_id,
		'url' : dataset_dict['resource_url'],
		'name' : dataset_dict['resource_name'],
		'description' : dataset_dict['resource_description'],
		'format' : dataset_dict['resource_format']
	}
	resource = get_resource_search_name_name(server, api_key, dataset_dict['resource_name'])
	if resource == None:
		send_request_to_server(server, api_key, 'resource_create', dataset_dict_resource)
	else:
		dataset_dict_resource['id'] = resource['id']
		send_request_to_server(server, api_key, 'resource_update', dataset_dict_resource)

## 4 Main code

In [ ]:
df = pd.read_csv(csv_file)
for index, row in df.iterrows():
	tags = row['Tags'].split(',')
	tag_list= []
	for tag in tags:
		tag_list.append({'name' : tag})
	dataset_dict = {
		'owner_org' : row['Organization'],
		'groups' : [
			{'id' : row['Group']}
		],
		'name' : row['Name'],
		'title' : row['Title'],
		'author' : row['Author'],
		'notes' : row['Notes'],
		'author_email' : row['AuthorEmail'],
		'maintainer' : row['Maintainer'],
		'maintainer_email' : row['MaintainerEmail'],
		'url' : row['Url'],
		'tags' : tag_list,
		'resource_name' : row['ResourceName'],
		'resource_url' : row['ResourceUrl'],
		'resource_format' : row['ResourceFormat'],
		'resource_description' : row['ResourceDescripiton'],
		'extras' : [
			{'key' : 'PublicAccess', 'value' : row['PublicAccess']},
			{'key' : 'UniversityAccess', 'value' : row['UniversityAccess']},
			{'key' : 'FacultyAccess', 'value' : row['FacultyAccess']},
			{'key' : 'DepartmentAccess', 'value' : row['DepartmentAccess']},
			{'key' : 'AdvisorAccess', 'value' : row['AdvisorAccess'] },
			{'key' : 'StudentAccess', 'value' : row['StudentAccess']},
			{'key' : 'PDPA', 'value' : row['PDPA']},
			{'key' : 'RetainPolicy', 'value' : row['ReatainPolicy'] },
			{'key' : 'UpdatePolicy', 'value' : row['UpdatePolicy'] }
		]
	}
	create_package(server, api_key, dataset_dict)
	create_resource(server, api_key, dataset_dict)